# 优化算法

矩阵乘法样例

In [ ]:
import numpy as np
import torch
import time

start = time.time()
A = torch.zeros((256,256))
B = torch.randn((256,256))
C = torch.randn((256,256))
# A = np.zeros((256,256))
# B = np.random.randn(256,256)
# C = np.random.randn(256,256)

for i in range(256):
    for j in range(256):
        A[i, j] = B[i, :] @ C[:, j]
        # A[i, j] = torch.dot(B[i, :] ,C[:, j])

end = time.time()
end - start

0.3318967819213867

In [ ]:
# 逐列计算
# torch.mv: matrix, vector
for j in range(256):
    A[:, j] = torch.mv(B, C[:, j])

## SGD随机梯度下降: Stochastic gradient desent
在GD中，每次迭代会用到**全部**训练数据。  
而在SGD中，每次迭代，对数据样本随机均匀采样一个索引$i$，只使用$样本_i$训练数据来更新参数。

## Minibatch GD 小批量随机梯度下降  
GD中使用完整数据集来计算梯度并更新参数， SGD中一次处理一个训练样本来取得进展。  
二者各有利弊：每当数据非常相似时，梯度下降并不是非常“数据高效”。 而由于CPU和GPU无法充分利用向量化，随机梯度下降并不特别“计算高效”。

小批量随机梯度下降就是折中方案。
$$\mathbf{g}_t = \partial_{\mathbf{w}} \frac{1}{|\mathcal{B}_t|} \sum_{i \in \mathcal{B}_t} f(\mathbf{x}_{i}, \mathbf{w})$$

## AdaGrad算法：
在训练过程中能够自动调整学习率。

## Adam算法  
* SGD：随机梯度下降在解决优化问题时比梯度下降更有效。  
* minibatch GD：在一个小批量中使用更大的观测值集，可以通过向量化提供额外效率。
* 动量法：添加了一种机制，用于汇总过去梯度的历史以加速收敛。
* AdaGard：我们使用每个坐标缩放来实现计算效率的预处理。
* Adadelta：我们通过学习率的调整来分离每个坐标的缩放。

Adam算法将这些技术汇总到一个高校的学习算法中。但是Adam并非没有问题，有时可能由于方差控制不良而发散。

1)
$$\begin{split}\begin{aligned}
    \mathbf{v}_t & \leftarrow \beta_1 \mathbf{v}_{t-1} + (1 - \beta_1) \mathbf{g}_t, \\
    \mathbf{s}_t & \leftarrow \beta_2 \mathbf{s}_{t-1} + (1 - \beta_2) \mathbf{g}_t^2.
\end{aligned}\end{split}$$

2)
$$\hat{\mathbf{v}}_t = \frac{\mathbf{v}_t}{1 - \beta_1^t} \text{ and } \hat{\mathbf{s}}_t = \frac{\mathbf{s}_t}{1 - \beta_2^t}.$$

3)
$$\mathbf{g}_t' = \frac{\eta \hat{\mathbf{v}}_t}{\sqrt{\hat{\mathbf{s}}_t} + \epsilon}.$$

4)
$$\mathbf{x}_t \leftarrow \mathbf{x}_{t-1} - \mathbf{g}_t'.$$

## 学习率调度器  

* 多项式衰减的一种替代方案是乘法衰减，即$\eta_{t+1} \leftarrow \eta_t \cdot \alpha$，为了防止学习率衰减超出合理的下限，更新方程经常修改为 $\eta_{t+1} \leftarrow \mathop{\mathrm{max}}(\eta_{\mathrm{min}}, \eta_t \cdot \alpha)$。  

* 保持分段稳定的学习率，而且每隔一段时间就一定程度降低学习率。 具体地说，给定一组降低学习率的时间，例如 𝑠={5,10,20} 每当 𝑡∈𝑠 时降低$\eta_{t+1} \leftarrow \eta_t \cdot \alpha$。
  
* 余弦调度器是 [Loshchilov & Hutter, 2016]提出的一种启发式算法。 它所依据的观点是：我们可能不想在一开始就太大地降低学习率，而且可能希望最终能用非常小的学习率来“改进”解决方案。 这产生了一个类似于余弦的调度，函数形式如下所示，学习率的值在 𝑡∈[0,𝑇] 之间。$$\eta_t = \eta_T + \frac{\eta_0 - \eta_T}{2} \left(1 + \cos(\pi t/T)\right)$$这里 𝜂0 是初始学习率， 𝜂𝑇 是当 𝑇 时的目标学习率。 此外，对于 𝑡>𝑇 ，我们只需将值固定到 𝜂𝑇 而不再增加它。 在下面的示例中，我们设置了最大更新步数 𝑇=20 。
  
* 预热。在训练初期将学习率增加至最大值。  
预热阶段限制了非常深的网络中参数的发散程度 。 这在直觉上是有道理的：在网络中那些一开始花费最多时间取得进展的部分，随机初始化会产生巨大的发散。

# NLP: 预训练

## 词嵌入 Word2Vec

将单词映射到向量的技术被称为词嵌入。词嵌入逐渐成为NLP的基础知识。

one-hot encoding很容易构建，但通常不是一个好的选择。一个主要的原因是它不能准确表达不同词之间的相似程度。 例如，“bank”（可以译作银行或者河岸）的向量表示在 “go to the bank to deposit some money”（去银行存点钱） 和“go to the bank to sit down”（去河岸坐下来）中是相同的。

word2vec工具是为了解决上述问题而提出的。它将每个词映射到一个固定长度的向量，这些向量能更好地表达不同词之间的相似性和类比关系。

word2vec工具包含两个模型，即跳元模型（skip-gram）和连续词袋（CBOW）。对于在语义上有意义的表示，它们的训练依赖于条件概率，条件概率可以被看作使用语料库中一些词来预测另一些单词。由于是不带标签的数据，因此跳元模型和连续词袋都是自监督模型。

- skip-gram: 跳元模型假设一个词可以用来在文本序列中生成其周围的单词。以文本序列“the”“man”“loves”“his”“son”为例。给定中心词“loves”，跳元模型考虑生成上下文词“the”“man”“him”“son”的条件概率  
  
- CBOW: 与跳元模型的主要区别在于，连续词袋模型假设中心词是基于其在文本序列中的周围上下文词生成的。例如，在文本序列“the”“man”“loves”“his”“son”中，在“loves”为中心词且上下文窗口为2的情况下，连续词袋模型考虑基于上下文词“the”“man”“him”“son”（如 图14.1.2所示）生成中心词“loves”的条件概率

## 应用预训练的词向量


### 加载预训练词向量

In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l

以下列出维度为50、100和300的预训练GloVe嵌入，可从GloVe网站下载。预训练的fastText嵌入有多种语言。这里我们使用可以从fastText网站下载300维度的英文版本（“wiki.en”）。

In [2]:
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

为了加载预训练的GloVe和fastText嵌入，定义一个TokenEmbedding类。

In [4]:
class TokenEmbedding:
    """GloVe嵌入"""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # 跳过标题信息，例如fastText中的首行
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [5]:
# 加载50维GloVe嵌入
glove_50d = TokenEmbedding('glove.6b.50d')

# 词表包含400000个词和一个特殊的未知词元
print(len(glove_50d))

400001


In [7]:
glove_50d.token_to_idx['<unk>'], glove_50d.idx_to_token[0]

(0, '<unk>')

In [9]:
glove_50d.idx_to_vec[0].shape

torch.Size([50])

## BERT

- word2vec和GloVe等词嵌入模型与上下文无关。它们将相同的预训练向量赋给同一个词，而不考虑词的上下文（如果有的话）。它们很难处理好自然语言中的一词多义或复杂语义。
  
- 对于上下文敏感的词表示，如ELMo和GPT，词的表示依赖于它们的上下文。
  
- ELMo对上下文进行双向编码，但使用特定于任务的架构（然而，为每个自然语言处理任务设计一个特定的体系架构实际上并不容易）；而GPT是任务无关的，但是从左到右编码上下文。
  
- BERT结合了这两个方面的优点：它对上下文进行双向编码，并且需要对大量自然语言处理任务进行最小的架构更改。
  
- BERT输入序列的嵌入是词元嵌入、片段嵌入和位置嵌入的和。
  
- 预训练包括两个任务：掩蔽语言模型和下一句预测。前者能够编码双向上下文来表示单词，而后者则显式地建模文本对之间的逻辑关系。



[BERT](./BERT/)

# NLP: 应用

NLP的任务流程图：
<center><img src='https://zh-v2.d2l.ai/_images/nlp-map-app.svg' /><center/>